In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import log_loss,accuracy_score

1. **MUST RUN 01-Data_Preprocessing/01-preprocessing_code.py AND 02-Feature_Engineering/01-fe_code.py BEFORE**

### We have the following dataset :

In [2]:
# load objects/X_DATASET.pkl and objects/Y_DATASET.pkl
X = pd.read_pickle('../objects/X_DATASET.pkl')
Y = pd.read_pickle('../objects/Y_DATASET.pkl')

In [3]:
X

,age,aliq_at,aliq_mat,ami_126d,at_be,at_gr1,at_me,at_turnover,be_gr1a,be_me,...,tax_gr1a,turnover_126d,turnover_var_126d,z_score,zero_trades_126d,zero_trades_21d,zero_trades_252d,log_diff,frac_diff,sadf
0,132,0.985818,0.059472,0.000026,1.260918,0.651395,0.042021,1.028183,0.310450,0.033325,...,0.018174,0.003022,0.336775,43.032362,0.004157,0.005002,0.003226,0.000000,0.000000,0.000000
23522,600,0.610528,0.069305,0.000105,2.318300,-0.088342,0.149204,0.845524,-0.035351,0.064359,...,-0.013003,0.002484,0.472298,10.113916,0.004944,0.006133,0.004822,0.000000,0.000000,0.000000
75824,325,NaN,NaN,0.004628,11.225380,0.119811,4.214064,0.086138,0.003968,0.375405,...,0.001342,0.003168,0.729098,NaN,0.003974,0.004473,0.004011,0.000000,0.000000,0.000000
76014,60,0.616255,0.219590,0.000887,2.971011,0.093083,0.441731,1.506417,0.083045,0.148680,...,0.002257,0.003886,0.509687,5.348204,0.003278,0.004529,0.003185,0.000000,0.000000,0.000000
129371,72,0.613501,0.109210,0.001278,2.574726,0.012174,0.169081,1.197614,0.235894,0.065669,...,0.029722,0.006013,1.056336,8.349836,0.001912,0.002674,0.002260,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77686,654,0.348917,0.075573,0.000042,4.048223,0.026266,0.219966,0.381231,0.051937,0.054336,...,-0.012389,0.003985,0.431750,5.694486,0.007268,0.005775,0.007261,0.171796,1.194481,-1.250080
110708,875,0.541670,0.469106,0.000079,3.390920,0.101955,2.579604,0.224324,0.020290,0.760739,...,0.000223,0.012001,0.473572,0.648063,0.002548,0.001582,0.002992,0.018898,0.624563,-0.776705
77970,683,0.412350,0.106346,0.000052,23.971774,-0.034589,0.334292,1.221020,-0.062910,0.013945,...,-0.009924,0.011881,0.590305,3.961255,0.002585,0.001735,0.003006,0.197150,1.228235,-1.518445
9565,527,0.566360,0.404144,0.000205,5.153496,0.190384,1.926307,2.614588,-0.023169,0.373786,...,-0.006986,0.013285,0.537003,3.973304,0.002241,0.002721,0.002129,0.039123,1.378815,-1.278062


In [4]:
Y

,stock_exret,target,prediction,probability,t1,t1_index,weight_attr
0,0.018070,1,1.0,0.364712,2000-01-31,2000-01-03,0.018070
23522,0.001539,1,-1.0,0.144033,2000-01-31,2000-01-03,0.001539
75824,0.054724,1,1.0,0.405740,2000-01-31,2000-01-03,0.054724
76014,0.009531,1,-1.0,0.533961,2000-01-31,2000-01-03,0.009531
129371,0.389768,1,-1.0,0.157432,2000-01-31,2000-01-03,0.389768
...,...,...,...,...,...,...,...
77686,0.065845,1,-1.0,0.661575,2023-12-29,2023-12-01,0.065845
110708,0.031191,1,1.0,0.186675,2023-12-29,2023-12-01,0.031191
77970,-0.009602,-1,1.0,0.822283,2023-12-29,2023-12-01,0.009602
9565,0.105924,1,1.0,0.973849,2023-12-29,2023-12-01,0.105924


In [5]:
X['t1_index'] = Y['t1_index']
X.reset_index(inplace=True)
X.set_index(['t1_index', 'index'], inplace=True)
X

age   aliq_at  aliq_mat  ami_126d      at_be    at_gr1  \
t1_index   index                                                            
2000-01-03 0       132  0.985818  0.059472  0.000026   1.260918  0.651395   
           23522   600  0.610528  0.069305  0.000105   2.318300 -0.088342   
           75824   325       NaN       NaN  0.004628  11.225380  0.119811   
           76014    60  0.616255  0.219590  0.000887   2.971011  0.093083   
           129371   72  0.613501  0.109210  0.001278   2.574726  0.012174   
...                ...       ...       ...       ...        ...       ...   
2023-12-01 77686   654  0.348917  0.075573  0.000042   4.048223  0.026266   
           110708  875  0.541670  0.469106  0.000079   3.390920  0.101955   
           77970   683  0.412350  0.106346  0.000052  23.971774 -0.034589   
           9565    527  0.566360  0.404144  0.000205   5.153496  0.190384   
           131463  936  0.502752  0.448166  0.000064   2.224959  0.033296   

                      at_me  at_turnover   be_gr1a     be_me  ...  tax_gr1a  \
t1_index   index                                              ...             
2000-01-03 0       0.042021     1.028183  0.310450  0.033325  ...  0.018174   
           23522   0.149204     0.845524 -0.035351  0.064359  ... -0.013003   
           75824   4.214064     0.086138  0.003968  0.375405  ...  0.001342   
           76014   0.441731     1.506417  0.083045  0.148680  ...  0.002257   
           129371  0.169081     1.197614  0.235894  0.065669  ...  0.029722   
...                     ...          ...       ...       ...  ...       ...   
2023-12-01 77686   0.219966     0.381231  0.051937  0.054336  ... -0.012389   
           110708  2.579604     0.224324  0.020290  0.760739  ...  0.000223   
           77970   0.334292     1.221020 -0.062910  0.013945  ... -0.009924   
           9565    1.926307     2.614588 -0.023169  0.373786  ... -0.006986   
           131463  1.989022     0.229889 -0.000261  0.893959  ...  0.000052   

                   turnover_126d  turnover_var_126d    z_score  \
t1_index   index                                                 
2000-01-03 0            0.003022           0.336775  43.032362   
           23522        0.002484           0.472298  10.113916   
           75824        0.003168           0.729098        NaN   
           76014        0.003886           0.509687   5.348204   
           129371       0.006013           1.056336   8.349836   
...                          ...                ...        ...   
2023-12-01 77686        0.003985           0.431750   5.694486   
           110708       0.012001           0.473572   0.648063   
           77970        0.011881           0.590305   3.961255   
           9565         0.013285           0.537003   3.973304   
           131463       0.007469           0.391692   1.081265   

                   zero_trades_126d  zero_trades_21d  zero_trades_252d  \
t1_index   index                                                         
2000-01-03 0               0.004157         0.005002          0.003226   
           23522           0.004944         0.006133          0.004822   
           75824           0.003974         0.004473          0.004011   
           76014           0.003278         0.004529          0.003185   
           129371          0.001912         0.002674          0.002260   
...                             ...              ...               ...   
2023-12-01 77686           0.007268         0.005775          0.007261   
           110708          0.002548         0.001582          0.002992   
           77970           0.002585         0.001735          0.003006   
           9565            0.002241         0.002721          0.002129   
           131463          0.004487         0.003086          0.004744   

                   log_diff  frac_diff      sadf  
t1_index   index                                  
2000-01-03 0       0.000000   0.000000  0.000000  
           235

### X contains the feautes given and ADDED BY US

In [6]:
Y = Y.reset_index()
Y.set_index(['t1_index', 'index'], inplace=True)
Y


stock_exret  target  prediction  probability          t1  \
t1_index   index                                                              
2000-01-03 0          0.018070       1         1.0     0.364712  2000-01-31   
           23522      0.001539       1        -1.0     0.144033  2000-01-31   
           75824      0.054724       1         1.0     0.405740  2000-01-31   
           76014      0.009531       1        -1.0     0.533961  2000-01-31   
           129371     0.389768       1        -1.0     0.157432  2000-01-31   
...                        ...     ...         ...          ...         ...   
2023-12-01 77686      0.065845       1        -1.0     0.661575  2023-12-29   
           110708     0.031191       1         1.0     0.186675  2023-12-29   
           77970     -0.009602      -1         1.0     0.822283  2023-12-29   
           9565       0.105924       1         1.0     0.973849  2023-12-29   
           131463     0.042407       1        -1.0     0.362189  2023-12-29   

                   weight_attr  
t1_index   index                
2000-01-03 0          0.018070  
           23522      0.001539  
           75824      0.054724  
           76014      0.009531  
           129371     0.389768  
...                        ...  
2023-12-01 77686      0.065845  
           110708     0.031191  
           77970      0.009602  
           9565       0.105924  
           131463     0.042407  

[131464 rows x 6 columns]

### Currently, in $Y$,

*prediction* and *probability* are randomly generated, those are the columns
we must fill for the investment perdiod (2008-) we our true probabilities and predictions.

## WE WANT TO PREDICT TARGET, which is simply the sign of stock_exret

## Weight_attr IS SO IMPORANT, PLEASE DON'T IGNORE 

As Uday did in feature importance, this weight_attr must be rescaled for the 
training period. Right now, it is scaled for the whole dataset, you must reajust
it for the training period such that the suym of the weight = Size of the training period

*= X_train.shape[0]/X_train['weight_attr'].sum()

Uday did it I believe

Refer to section 4.6 if you have any doubts

# Hyperparameter Tuning

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical

# Define base RandomForestClassifier (without setting n_estimators because we'll search for it)
base_rf = RandomForestClassifier(
    criterion="entropy",  # use entropy for information gain
    bootstrap=False,  # no bootstrap
    class_weight="balanced_subsample"  # handle class imbalance
)

# Define the parameter search space for BayesSearchCV
param_space = {
    'estimator__n_estimators': Integer(10, 100),  # number of trees in RF
    'estimator__max_depth': Integer(5, 50),  # depth of each tree
    'estimator__min_samples_split': Integer(2, 10),  # minimum samples required to split a node
    'estimator__min_samples_leaf': Integer(1, 5),  # minimum samples required in a leaf node
    'estimator__max_features': Categorical(["auto", "sqrt", "log2"]),  # feature selection for RF
    'n_estimators': Integer(10, 500),  # number of bagging classifiers
    'max_samples': Real(0.1, 1.0),  # fraction of samples for each bagging classifier
    'max_features': Real(0.1, 1.0),  # fraction of features for each base estimator
}

# Initialize BaggingClassifier with RandomForestClassifier as the base estimator
bagging_clf = BaggingClassifier(
    estimator=base_rf,
    oob_score=True,
    n_jobs=-1
)

# Set up BayesSearchCV with BaggingClassifier and the parameter search space
optimizer = BayesSearchCV(
    bagging_clf,
    param_space,
    n_iter=50,  # number of iterations for hyperparameter search
    cv=3,  # 3-fold cross-validation
    n_jobs=-1,
    verbose=1
)

# Fit the model with training data
optimizer.fit(X_train, Y_train)

# Get the best parameters and model
best_model = optimizer.best_estimator_
print(f"Best parameters found: {optimizer.best_params_}")

# Make predictions using the optimized model
predictions = best_model.predict(X_test)

## Method

The method is explained in the FIAM instructions, please refer to pages\
6 and 7, sections **Performance evaluation** AND **Training Procedures**\
(especially training procedures)

As they mention it in the doc, they provided an implementation of the method,

penalized_linear_hackathon.py

In [14]:
# Necessary imports
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Integer, Real, Categorical
from scipy.stats import randint, uniform

# Initialize the starting date, counter, and output data
stock_vars = ['ret_1_0', 'prc_highprc_252d', 'seas_1_1na', 'rmax5_21d']  # List of features
tgt_var = 'target'  # Target variable

starting = pd.to_datetime("20000101", format="%Y%m%d")
counter = 0
pred_out = pd.DataFrame()



# Ensure that the indices of X and Y are MultiIndex with datetime as the first level
X.index = pd.MultiIndex.from_tuples(
    [(pd.to_datetime(t1_index), other_index) for t1_index, other_index in X.index]
)
Y.index = pd.MultiIndex.from_tuples(
    [(pd.to_datetime(t1_index), other_index) for t1_index, other_index in Y.index]
)

# Estimation with expanding window
while (starting + pd.DateOffset(years=11 + counter)) <= pd.to_datetime("20240101", format="%Y%m%d"):
    # For testing purposes, limit the number of iterations
    # Remove or adjust this condition as needed
    if counter == 1:
        break

    cutoff = [
        starting,
        starting + pd.DateOffset(years=8 + counter),  # Training set end date
        starting + pd.DateOffset(years=10 + counter),  # Validation set end date
        starting + pd.DateOffset(years=11 + counter),  # Test set end date
    ]

    print(f"Train Start: {cutoff[0]}, Train End: {cutoff[1]}, Val Start: {cutoff[1]}, Val End: {cutoff[2]}, Test Start: {cutoff[2]}, Test End: {cutoff[3]}")

    # Cut the sample into training, validation, and testing sets
    X_train = X[(X.index.get_level_values(0) >= cutoff[0]) & (X.index.get_level_values(0) < cutoff[1])]
    X_validate = X[(X.index.get_level_values(0) >= cutoff[1]) & (X.index.get_level_values(0) < cutoff[2])]
    X_test = X[(X.index.get_level_values(0) >= cutoff[2]) & (X.index.get_level_values(0) < cutoff[3])]

    Y_train = Y[(Y.index.get_level_values(0) >= cutoff[0]) & (Y.index.get_level_values(0) < cutoff[1])]
    Y_validate = Y[(Y.index.get_level_values(0) >= cutoff[1]) & (Y.index.get_level_values(0) < cutoff[2])]
    Y_test = Y[(Y.index.get_level_values(0) >= cutoff[2]) & (Y.index.get_level_values(0) < cutoff[3])]

    # Adjust sample weights
    Y_train['weight_attr'] *= Y_train.shape[0] / Y_train['weight_attr'].sum()
    Y_validate['weight_attr'] *= Y_validate.shape[0] / Y_validate['weight_attr'].sum()
    Y_test['weight_attr'] *= Y_test.shape[0] / Y_test['weight_attr'].sum()

    # Get feature matrices
    X_train_vals = X_train[stock_vars].values
    X_validate_vals = X_validate[stock_vars].values
    X_test_vals = X_test[stock_vars].values

    # Combine training and validation sets
    X_train_val = np.vstack([X_train_vals, X_validate_vals])
    Y_train_val = pd.concat([Y_train, Y_validate])

    # Create test_fold for PredefinedSplit
    test_fold = np.concatenate([
        np.full(len(X_train_vals), -1),  # Training set indices
        np.zeros(len(X_validate_vals))   # Validation set indices
    ])
    ps = PredefinedSplit(test_fold)

    # Define base estimator
    base_rf = RandomForestClassifier(
        criterion="entropy",
        bootstrap=False,
        class_weight="balanced_subsample"
    )

    # Define bagging classifier
    bagging_clf = BaggingClassifier(
        estimator=base_rf,
        oob_score=True,
        n_jobs=-1
    )

    # Define parameter space for hyperparameter optimization
    param_distributions = {
    'estimator__n_estimators': randint(10, 1000),
    'estimator__max_depth': randint(5, 50),
    'estimator__min_samples_split': randint(2, 10),
    'estimator__min_samples_leaf': randint(1, 5),
    'estimator__max_features': ['sqrt', 'log2'],  # Categorical choices for RandomForestClassifier
    'n_estimators': randint(10, 100),  # Number of bagging estimators
    'max_samples': uniform(0.1, 1.0),  # Fraction of samples for each bagging classifier
    'max_features': uniform(0.1, 1.0)  # Fraction of features for each bagging classifier (must be a float)
    }

    # Define the optimizer
    optimizer = RandomizedSearchCV(
    bagging_clf,
    param_distributions=param_distributions,
    n_iter=50,  # Number of iterations for hyperparameter search
    cv=ps,      # Use predefined split
    n_jobs=-1,
    verbose=2,
    random_state=42
    )   

    # Fit optimizer with sample weights
    optimizer.fit(
        X_train_val,
        Y_train_val[tgt_var].values,
        **{'sample_weight': Y_train_val['weight_attr'].values}
    )

    # Retrieve the best estimator
    best_estimator = optimizer.best_estimator_

    # Retrain best_estimator on combined training and validation set
    best_estimator.fit(
        X_train_val,
        Y_train_val[tgt_var].values,
        sample_weight=Y_train_val['weight_attr'].values
    )

    # Predict on test set
    prob = best_estimator.predict_proba(X_test_vals)
    score_ = -log_loss(Y_test[tgt_var].values, prob, sample_weight=Y_test["weight_attr"].values, labels=best_estimator.classes_)
    print("Log Loss on Test Set:", score_)

    # Store predictions in Y_test
    Y_test['prediction'] = best_estimator.predict(X_test_vals)
    Y_test['probability'] = prob.max(axis=1)

    print("Predictions on Test Set:")
    print(Y_test['prediction'])
    print("Probabilities on Test Set:")
    print(Y_test['probability'])
    print("Prediction Probabilities:")
    print(prob)

    # Optionally, store predictions in pred_out DataFrame
    # pred_out = pred_out.append(Y_test[['prediction', 'probability']])

    # Increment counter
    counter += 1


Train Start: 2000-01-01 00:00:00, Train End: 2008-01-01 00:00:00, Val Start: 2008-01-01 00:00:00, Val End: 2010-01-01 00:00:00, Test Start: 2010-01-01 00:00:00, Test End: 2011-01-01 00:00:00
Fitting 1 folds for each of 50 candidates, totalling 50 fits
[CV] END estimator__max_depth=6, estimator__max_features=log2, estimator__min_samples_leaf=4, estimator__min_samples_split=7, estimator__n_estimators=815, max_features=0.10077876584101433, max_samples=1.0922115592912176, n_estimators=42; total time=   0.0s


/var/folders/r_/jx86zm6j6jz70fwxkrhjzcfw0000gn/T/ipykernel_32806/1966811691.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_train['weight_attr'] *= Y_train.shape[0] / Y_train['weight_attr'].sum()
/var/folders/r_/jx86zm6j6jz70fwxkrhjzcfw0000gn/T/ipykernel_32806/1966811691.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Y_validate['weight_attr'] *= Y_validate.shape[0] / Y_validate['weight_attr'].sum()
/var/folders/r_/jx86zm6j6jz70fwxkrhjzcfw0000gn/T/ipykernel_32806/1966811691.py:59: SettingWithCop

[CV] END estimator__max_depth=7, estimator__max_features=sqrt, estimator__min_samples_leaf=3, estimator__min_samples_split=8, estimator__n_estimators=30, max_features=0.550499251969543, max_samples=0.11326496115986653, n_estimators=98; total time=  43.8s


## THE CLASSIFIER

You can use one of the following. For clarification, refer to **6.4**

I believe clf2 is the best one

### This is from my deprado repo, *ch_06.ipynb*

```python
# Libraries
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dataset
X, Y = f_ch8.getTestData(n_features=40, n_informative=3, n_redundant=30, n_samples=10000)
Y = Y["bin"]

# usual RF
clf0 = RandomForestClassifier(
    n_estimators=1_000,  # 1_000 trees
    class_weight="balanced_subsample",  # prevent minority class from being ignored
    criterion="entropy"  # information gain
)

# Ensemble of estimators with base estimator as a decision tree
clf1 = DecisionTreeClassifier(
    criterion="entropy",  # information gain
    max_features="sqrt",  # sqrt(n_features) to force diversity among trees
    class_weight="balanced"  # prevent minority class from being ignored
)
clf1 = BaggingClassifier(
    estimator=clf1,  # base estimator
    n_estimators=1_000,  # 1_000 trees
    max_samples=0.6,  # average uniqueness
    max_features=1.0  # all features for bagging
)

# Bagging classifier on RF where max_samples is set to average uniqueness
clf2 = RandomForestClassifier(
    n_estimators=1,  # 1 tree
    criterion="entropy",  # information gain
    bootstrap=False,  # no bootstrap
    class_weight="balanced_subsample"  # prevent minority class from being ignored
)

clf2 = BaggingClassifier(
    estimator=clf2,  # base estimator
    n_estimators=1_000,  # 1_000 trees
    max_samples=1.0,  # average uniqueness
    max_features=1.0  # all features for bagging
)
```

## Hyperparameter Tunning

### Here is why the weights are so important



Refer to *9.4* , you can use accuracy, but negative log likelihood is suggested,\
since it takes into account the probability of the prediction.

For the hyperparaemtes, i believe you guys can figure out better than me what
to set

## Please be aware of the bugs mentioned in

- 7.5 BUGS IN SKLEARN’S CROSS-VALIDATION
- SNIPPET 9.2 AN ENHANCED PIPELINE CLASS


## I'M AVAILABLE ALL DAY, PLEASE ASK ME ANYTHING, LET'S GOOOOO